In [1]:
from hail import *
hc = HailContext(tmp_dir = '/nvme/tmp/shusson')

Running on Apache Spark version 2.1.0
SparkUI available at http://129.94.72.55:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.1-c72c76d


In [2]:
vds = hc.read('/nvme/tmp/shusson/small_300.vds')

In [3]:
vds.summarize()

Summary(samples=2845, variants=74200, call_rate=0.991872, contigs=['X', '12', '8', '19', '4', '15', '11', '9', 'Y', '22', '13', '16', '5', '10', '21', '6', '1', '17', '14', 'MT', '20', '2', '18', '7', '3'], multiallelics=6232, snps=66570, mnps=0, insertions=8307, deletions=9746, complex=0, star=3564, max_alleles=44)

In [3]:
vds = vds.split_multi()

In [4]:
vds = vds.annotate_variants_expr([
        'va.allele = va.alleles[va.aIndex-1]',
        'va.extra.wasSplit = va.wasSplit',
        'va.extra.nHomVarFemale = gs.filter(s => sa.pheno.isFemale).filter(g => g.isHomVar()).count()',
        'va.extra.nHomVarMale = gs.filter(s => !sa.pheno.isFemale).filter(g => g.isHomVar()).count()',
        'va.extra.nHomRefFemale = gs.filter(s => sa.pheno.isFemale).filter(g => g.isHomRef()).count()',
        'va.extra.nHomRefMale = gs.filter(s => !sa.pheno.isFemale).filter(g => g.isHomRef()).count()'
    ]).annotate_variants_expr([
        'va.extra.nHomVar = va.extra.nHomVarMale + va.extra.nHomVarFemale',
        'va.extra.nHomRef = va.extra.nHomRefMale + va.extra.nHomRefFemale'
    ])

In [5]:
vds = vds.filter_variants_expr('va.allele.metrics.allele_counts.total.toInt() <= 0', keep=False)

In [6]:
vds = vds.annotate_variants_expr('''
    va.extra.nHet = {
        total: 
            if (v.inXNonPar())
                (2 * (va.allele.metrics.allele_counts.male - va.extra.nHomVarMale)) + (va.allele.metrics.allele_counts.female - (2 * va.extra.nHomVarFemale))
            else if (v.inYNonPar())
                2 * (va.allele.metrics.allele_counts.total - va.extra.nHomVar)
            else
                va.allele.metrics.allele_counts.total - (2 * va.extra.nHomVar)   
            }
    ''')


In [7]:
vds.write('/nvme/tmp/shusson/sample_stats.vds')